In [ ]:
# pip install pyLDAvis

https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/nlp/topic-modeling-naive.html

In [58]:
import pandas as pd
import numpy as np
from deep_translator import GoogleTranslator, MyMemoryTranslator
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from langdetect import detect
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE
import pyLDAvis

import re

In [18]:
df = pd.read_csv("scraped_2.csv", index_col=False)

In [19]:
df.columns

Index(['Unnamed: 0', 'Article', 'URL'], dtype='object')

In [20]:
df = df.drop(columns=['Unnamed: 0'])

In [21]:
df.head()

,Article,URL
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...


In [22]:
# pip install langdetect

In [25]:
def translate(text):
#     try:
#         language = detect(text)
#     except:
#         language = "Unknown"
#     if language == 'en':
#         trans_sent = text
        
#     else:  
    text = text[:1500]
    trans_sent = GoogleTranslator(source='auto', target='en').translate(text)
    
    return trans_sent

In [26]:
df['Article_trans'] = df['Article'].apply(lambda x: translate(x))

In [27]:
df.head()

,Article,URL,Article_trans
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...,The UN Office for the Coordination of Humanita...
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...,The Australia and New Zealand Banking Group Li...
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...,The Myanmar Military Council's election commis...
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...,Human Rights Watch (HRW) Myanmar appeared to b...
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...,"In Monywa, Sagaing Province, the car was attac..."


In [28]:
df.iloc[4,2]

'In Monywa, Sagaing Province, the car was attacked by a mine because it was the car of the coup leader Min Aung Hlaing, but it was the car carrying the police chief, and 4 deputy police chiefs were killed.\n \nCopyright © 2023 All rights reserved by'

In [30]:
# df.to_csv('scraped_4_translated.csv')

# Data Preprocessing

In [31]:
# Lowercasing
df['Article_lower'] = df['Article_trans'].apply(lambda x: x.lower())

In [32]:
df.head()

,Article,URL,Article_trans,Article_lower
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...,The UN Office for the Coordination of Humanita...,the un office for the coordination of humanita...
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...,The Australia and New Zealand Banking Group Li...,the australia and new zealand banking group li...
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...,The Myanmar Military Council's election commis...,the myanmar military council's election commis...
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...,Human Rights Watch (HRW) Myanmar appeared to b...,human rights watch (hrw) myanmar appeared to b...
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...,"In Monywa, Sagaing Province, the car was attac...","in monywa, sagaing province, the car was attac..."


In [33]:
# Tokenizing
df['Article_tokenized'] = df['Article_lower'].apply(lambda x: word_tokenize(x))

In [34]:
df.head()

,Article,URL,Article_trans,Article_lower,Article_tokenized
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...,The UN Office for the Coordination of Humanita...,the un office for the coordination of humanita...,"[the, un, office, for, the, coordination, of, ..."
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...,The Australia and New Zealand Banking Group Li...,the australia and new zealand banking group li...,"[the, australia, and, new, zealand, banking, g..."
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...,The Myanmar Military Council's election commis...,the myanmar military council's election commis...,"[the, myanmar, military, council, 's, election..."
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...,Human Rights Watch (HRW) Myanmar appeared to b...,human rights watch (hrw) myanmar appeared to b...,"[human, rights, watch, (, hrw, ), myanmar, app..."
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...,"In Monywa, Sagaing Province, the car was attac...","in monywa, sagaing province, the car was attac...","[in, monywa, ,, sagaing, province, ,, the, car..."


In [36]:
# Stopwords removal
stop_words_own = ["due","june","people","hours","three", "wednesday", "group","moe","pdfs","day","now", "long","issued","year",
       "dont","daw","nug","san","including","kyi","crn","nld","told","first",
       "suu","february","well","years", "coup","said","days","two","will","also","new","min","since","nug",
       "“","daw","even","”", "wwwmizzimacomwwwmizzimaburmesecomwwwmizzimaweeklycom","email protectedemail protected","special",
       "kia", "fight","army","forces","curfew","shot", "war","burned","police","camp", "fire","militia","attack",
       "van","around","back", "behind","hall","one","saw","opened", "bin","swe","went","side","less",
       "come","copyright","reserved", "october","–","today","ppdb","doesnt", "made","get","use","take",
       "ready","still", "see","via","visit","dpd","pkb", "’s","four","least","last","former","can","five","”","six",
       "high", "–","near","top","set", "many", "email protectedemail protected", "voa","like","march","photos",
       "hour", "time","amp","show","every","main","things"]
def stop_words_rm(word_tokens):
    stop_words = stopwords.words('english') + stop_words_own
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

In [37]:
df['Article_stop_wd_rm'] = df['Article_tokenized'].apply(lambda x: stop_words_rm(x))

In [38]:
df.head()

,Article,URL,Article_trans,Article_lower,Article_tokenized,Article_stop_wd_rm
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...,The UN Office for the Coordination of Humanita...,the un office for the coordination of humanita...,"[the, un, office, for, the, coordination, of, ...","[un, office, coordination, humanitarian, affai..."
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...,The Australia and New Zealand Banking Group Li...,the australia and new zealand banking group li...,"[the, australia, and, new, zealand, banking, g...","[australia, zealand, banking, limited, (, anz,..."
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...,The Myanmar Military Council's election commis...,the myanmar military council's election commis...,"[the, myanmar, military, council, 's, election...","[myanmar, military, council, 's, election, com..."
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...,Human Rights Watch (HRW) Myanmar appeared to b...,human rights watch (hrw) myanmar appeared to b...,"[human, rights, watch, (, hrw, ), myanmar, app...","[human, rights, watch, (, hrw, ), myanmar, app..."
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...,"In Monywa, Sagaing Province, the car was attac...","in monywa, sagaing province, the car was attac...","[in, monywa, ,, sagaing, province, ,, the, car...","[monywa, ,, sagaing, province, ,, car, attacke..."


In [39]:
# Lematizing
def lem(stwd_rm_lst):
    lem_sent = []
    wnl = WordNetLemmatizer()
    for word in stwd_rm_lst:
        word = wnl.lemmatize(word, pos="n")
        lem_sent.append(word) 
    return lem_sent

In [40]:
df['lem_article'] = df['Article_stop_wd_rm'].apply(lambda x: lem(x))

In [41]:
df.head()

,Article,URL,Article_trans,Article_lower,Article_tokenized,Article_stop_wd_rm,lem_article
0,\nThe UN Office for the Coordination of Humani...,https://mizzima.com/article/un-and-asean-risk-...,The UN Office for the Coordination of Humanita...,the un office for the coordination of humanita...,"[the, un, office, for, the, coordination, of, ...","[un, office, coordination, humanitarian, affai...","[un, office, coordination, humanitarian, affai..."
1,\nThe Australia and New Zealand Banking Group ...,https://mizzima.com/article/anz-will-become-fi...,The Australia and New Zealand Banking Group Li...,the australia and new zealand banking group li...,"[the, australia, and, new, zealand, banking, g...","[australia, zealand, banking, limited, (, anz,...","[australia, zealand, banking, limited, (, anz,..."
2,\nThe Myanmar Military Council's election comm...,https://mizzima.com/article/myanmar-junta-says...,The Myanmar Military Council's election commis...,the myanmar military council's election commis...,"[the, myanmar, military, council, 's, election...","[myanmar, military, council, 's, election, com...","[myanmar, military, council, 's, election, com..."
3,\nHuman Rights Watch (HRW) Myanmar appeared to...,https://mizzima.com/article/controversial-ngo-...,Human Rights Watch (HRW) Myanmar appeared to b...,human rights watch (hrw) myanmar appeared to b...,"[human, rights, watch, (, hrw, ), myanmar, app...","[human, rights, watch, (, hrw, ), myanmar, app...","[human, right, watch, (, hrw, ), myanmar, appe..."
4,\nစစ်ကိုင်းတိုင်း မုံရွာမြို့မှာ အာဏာသိမ်းစစ်ခ...,https://mizzima.com/article/independent-myanma...,"In Monywa, Sagaing Province, the car was attac...","in monywa, sagaing province, the car was attac...","[in, monywa, ,, sagaing, province, ,, the, car...","[monywa, ,, sagaing, province, ,, car, attacke...","[monywa, ,, sagaing, province, ,, car, attacke..."


In [42]:
df['lem_article_join'] = df['lem_article'].apply(lambda x: ' '.join(x))

In [43]:
df['lem_article_join'] = df['lem_article_join'].str.replace(r'\d+', '', regex=True)

# Feature Extraction

In [63]:
# v = TfidfVectorizer()
# x = v.fit_transform(df['lem_article_join'])
# vec = x.toarray()

cv = CountVectorizer(min_df=0., max_df=1., stop_words='english')
cv_matrix = cv.fit_transform(df['lem_article_join'])
cv_matrix

<1403x14076 sparse matrix of type '<class 'numpy.int64'>'
	with 114251 stored elements in Compressed Sparse Row format>

In [64]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
vocab = cv.get_feature_names_out()

# Latent Dirichlet Allocation

In [49]:
lda = LatentDirichletAllocation(n_components=2, max_iter=10000, random_state=0)
doc_topic_matrix = lda.fit_transform(cv_matrix)

In [53]:
# log-likelihood
print(lda.score(cv_matrix))
# perplexity
print(lda.perplexity(cv_matrix))

-1296968.0345390039
2666.889538368324


In [54]:
# Document-by-Topic Matrix
doc_topic_df = pd.DataFrame(doc_topic_matrix, columns=['T1', 'T2'])
doc_topic_df

,T1,T2
0,0.994770,0.005230
1,0.994716,0.005284
2,0.994682,0.005318
3,0.994977,0.005023
4,0.968204,0.031796
...,...,...
1398,0.991948,0.008052
1399,0.994258,0.005742
1400,0.970411,0.029589
1401,0.993420,0.006580


In [55]:
# Topic-by-Word Matrix
topic_word_matrix = lda.components_

In [69]:
pd.DataFrame(topic_word_matrix, columns=vocab)
pd.DataFrame(np.transpose(topic_word_matrix), index=vocab)

,0,1
_ym,1.488563,0.511437
aa,113.978220,2.021780
aacc,4.499701,0.500299
aapp,2.499194,0.500806
ab,0.500235,12.499765
...,...,...
zuna,2.499523,0.500477
zwa,2.499637,0.500363
zwekabin,1.499668,0.500332
zyun,1.499467,0.500533


In [101]:
def get_topics_meanings(tw_m,
                        vocab,
                        display_weights=False,
                        topn = 5,
                        weight_cutoff=0.6):
    topic_list = []
    for i, topic_weights in enumerate(tw_m):  ## for each topic row
        topic = [(token, np.round(weight, 2))
                 for token, weight in zip(vocab, topic_weights)
                 ]  ## zip (word, importance_weight)
        topic = sorted(topic,
                       key=lambda x: -x[1])  ## rank words according to weights
        topic_topn = topic[:topn]
        topic_topn = [word for word, weight in topic_topn]
        topic_list.append(topic_topn)
    return topic_list

In [102]:
topics = get_topics_meanings(topic_word_matrix, vocab, display_weights=False, topn=200)
print(topics)

[['military', 'council', 'myanmar', 'township', 'village', 'aung', 'state', 'junta', 'according', 'air', 'national', 'battle', 'defense', 'country', 'regime', 'leader', 'killed', 'arrested', 'member', 'government', 'th', 'local', 'support', 'soldier', 'general', 'pdf', 'prison', 'resident', 'force', 'party', 'yangon', 'dvb', 'news', 'ko', 'armed', 'china', 'hlaing', 'old', 'irrawaddy', 'collins', 'karen', 'area', 'city', 'minister', 'car', 'region', 'weapon', 'month', 'troop', 'group', 'civilian', 'burma', 'win', 'house', 'came', 'democratic', 'reported', 'sagaing', 'morning', 'school', 'thailand', 'company', 'resistance', 'world', 'information', 'road', 'official', 'year', 'battalion', 'burmese', 'mandalay', 'injured', 'organization', 'death', 'border', 'political', 'fighting', 'body', 'voice', 'woman', 'election', 'july', 'august', 'kyat', 'announced', 'security', 'thai', 'died', 'foreign', 'commander', 'source', 'tun', 'division', 'unity', 'based', 'rakhine', 'mizzima', 'chief', 'ye

In [103]:
df2 = pd.DataFrame()
df2 = df2.append(pd.DataFrame([topics], 
     columns=["TOPIC_1","TOPIC_2"]), 
     ignore_index=False)

/var/folders/nm/0kqqdhdj795byzw17d7bdmmw0000gn/T/ipykernel_48311/655838812.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([topics],


In [104]:
df2

,TOPIC_1,TOPIC_2
0,"[military, council, myanmar, township, village...","[advertisement, jogja, read, com, selling, loo..."


In [105]:
df2.to_csv('topic_modeling_two_topiocs.csv')